In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from typing import NamedTuple
from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import macd_tv, ema_tv

np.set_printoptions(formatter = {'float_kind': "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange = 'mufex',
    symbol = 'BTCUSDT',
    timeframe = '5m',
    candles_to_dl = 1500
)

In [3]:
datetimes = candles[:, CandleBodyType.Timestamp].astype('datetime64[ms]')
closing_prices = candles[:, CandleBodyType.Close]
low_prices = candles[:, CandleBodyType.Low]

In [4]:
# Only generate signals when MACD is below 0
macd_below = 0
fast_length = 12
slow_length = 26
signal_smoothing = 9
ema_length = 200

In [5]:
histogram, macd, signal = macd_tv(
    source = closing_prices,
    fast_length = fast_length,
    slow_length = slow_length,
    signal_smoothing = signal_smoothing
)

In [6]:
ema = ema_tv(
    source = closing_prices,
    length = ema_length
)

In [7]:
# When the macd crosses above the signal
prev_macd = np.roll(macd, 1)
prev_macd[0] = np.nan

previous_signal = np.roll(signal, 1)
previous_signal[0] = np.nan

In [8]:
macd_below_signal = prev_macd < previous_signal
macd_above_signal = macd > signal
low_price_below_ema = low_prices > ema
macd_below_number = macd < macd_below

In [9]:
entries =(
    (low_price_below_ema == True)
    & (macd_below_signal == True)
    & (macd_above_signal == True)
    & (macd_below_number == True)
)

In [10]:
entry_signals = np.where(entries, macd, np.nan)

In [17]:
fig = go.Figure()
fig = make_subplots(
    cols = 1,
    rows = 2,
    shared_xaxes = True,
    subplot_titles = ['Candles', 'MACD'],
    row_heights = [0.6, 0.4],
    vertical_spacing = 0.1
)
# Candlstick chart for pricing
fig.append_trace(
    go.Candlestick(
        x = datetimes,
        open = candles[:, CandleBodyType.Open],
        high = candles[:, CandleBodyType.High],
        low = candles[:, CandleBodyType.Low],
        close = candles[:, CandleBodyType.Close],
        name = 'Candles',   
    ),
    col = 1,
    row = 1
)
fig.append_trace(
    go.Scatter(
        x = datetimes,
        y = ema,
        name = 'EMA',
        line_color = 'yellow'
    ),
    col = 1,
    row = 1
)
ind_shift = np.roll(histogram, 1)
ind_shift[0] = np.nan
colors = np.where(
    histogram > 0,
    np.where(ind_shift < histogram, 'green', 'lightgreen'),
    np.where(ind_shift < histogram, 'salmon', 'red')
)
fig.append_trace(
    go.Bar(
        x = datetimes,
        y = histogram,
        name = 'Histogram',
        marker_color = colors
    ),
    row = 2,
    col = 1
)
fig.append_trace(
    go.Scatter(
        x = datetimes,
        y = macd,
        name = 'MACD',
        line_color = 'blue'
    ),
    row = 2,
    col = 1
)
fig.append_trace(
    go.Scatter(
        x = datetimes,
        y = signal,
        name = 'Signal',
        line_color = 'orange'
    ),
    row = 2,
    col = 1
)
fig.append_trace(
    go.Scatter(
        x = datetimes,
        y = entry_signals,
        mode = 'markers',
        marker = dict(
            size = 12,
            symbol = 'triangle-up',
            color = 'green',
            line = dict(
                width = 2,
                color = 'black'
            )
        ),
    ),
    row = 2,
    col = 1
)
# Update options and show plot
fig.update_layout(height = 800, xaxis_rangeslider_visible = False)
fig.show()